In [89]:
from dataclasses import dataclass
from enum import Enum, auto
from typing import Self
from uuid import uuid4
from bamt.preprocessors import Preprocessor
import pandas as pd
from sklearn import preprocessing as pp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import re
YEAR = 2022

In [90]:
df = pd.read_csv(

    "/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/mig whereabouts/inflow LO.csv")

Выделим ОКТМО из Ленобласти

In [91]:
oktmos = map(str,  df.oktmo.unique())

In [92]:
oktmos_lo = [x for x in oktmos if re.fullmatch(r"41\d+", x)]

In [93]:
del oktmos

In [94]:
len(oktmos_lo)

226

Определим численность населения. Проблема: удалось найти данные только за 2024 год

In [95]:
pop_size_df = pd.read_excel(

    "/Users/pishchulov/edu/matobes/НИР/outer_source/численность МО/Сhisl_MO_01-01-2024_only_LO.xlsx")

In [96]:
pop_size_df

,Коды территорий,Unnamed: 1,население,городское,сельское
0,4100000000,Ленинградская область,2035762,1373533,662229
1,41754000 0 0,Сосновоборский городской округ,63462,63462,0
2,41754000001 1 0 0 0,г Сосновый Бор,63462,63462,0
3,41603000 0 0,Бокситогорский муниципальный район,50855,39107,11748
4,41603101 0 0,Городское поселение Бокситогорское,15764,15480,284
...,...,...,...,...,...
271,41648418 0 0,Сельское поселение Нурминское,3338,0,3338
272,41648430 0 0,Сельское поселение Лисинское,1875,0,1875
273,41648443 0 0,Сельское поселение Тельмановское,26781,0,26781
274,41648444 0 0,Сельское поселение Трубникоборское,1599,0,1599


In [97]:
def discard_to_8_chars(s):
    if len(s) != 10:
        return s
    if s[-2:] == "00":
        return s[:-2]

In [98]:
pop_size_df["Коды территорий"] = pd.Series([discard_to_8_chars(x.replace(

    " ", "")) for x in map(str, pop_size_df["Коды территорий"])], dtype=str)

In [99]:
# pop_size_df_lo = pop_size_df[pop_size_df["Коды территорий"].isin(oktmos_lo)]

In [100]:
# set(oktmos_lo) - set(pop_size_df_lo["Коды территорий"])

Проблема: некоторые ОКТМО исчезли, потому что они эти населенные пункту переехали на новые коды. Проигнорируем.

In [101]:
del pop_size_df

In [102]:
# pop_size_df_lo=pop_size_df_lo.drop(["городское", "сельское"], axis = 1)
# pop_size_df_lo=pop_size_df_lo.set_axis(["oktmo", "name", "pop_size"], axis=1)
# pop_size_df_lo.sort_values("oktmo", inplace=True)
# pop_size_df_lo

Удалим муниципальные районы, потому что это агрегаты других МО

In [103]:
def is_mun_district(oktmo):
    assert len(oktmo) == 8
    return oktmo[-3:] == "000"

In [104]:
# pop_size_df_lo.pop_size.sum()

In [105]:
# mun_districts_lo = pop_size_df_lo[pop_size_df_lo.oktmo.apply(is_mun_district)]
# mun_districts_lo

In [106]:
# pop_size_df_lo=pop_size_df_lo[~pop_size_df_lo.oktmo.apply(is_mun_district)]
# pop_size_df_lo.pop_size.sum()

### Подготовим данные о среде. 

In [107]:
allmun = pd.read_csv(

    "/Users/pishchulov/edu/matobes/НИР/outer_source/rosstat_allmun/allmuns/popsize (allmun).csv")
allmun[["oktmo"]] = allmun[["oktmo"]].astype(str)
allmun["from_lo"] = [bool(re.fullmatch(r"41\d+", x)) for x in allmun.oktmo]
allmun = allmun[(allmun.from_lo) & (allmun.year == 2022)]
allmun = allmun[["oktmo", "name", "year"]].set_index("oktmo")

In [108]:
allmun

,name,year
oktmo,,
41603000,Бокситогорский муниципальный район,2022
41603101,Бокситогорское,2022
41603102,Пикалёвское,2022
41603155,Ефимовское,2022
41603412,Большедворское,2022
...,...,...
41648430,Лисинское,2022
41648443,Тельмановское,2022
41648444,Трубникоборское,2022


Нет ITMO2 данных за 2023 год, поэтому используем 2022.

In [109]:

mo_features = allmun
# files = [
#     "shoparea",
#     "foodseats",
#     "agrprod",
#     "beforeschool",
#     "schoolnum",  # тут нет требуемых районов
#     "museums",
#     "theatres",
#     "musartschool",
#     "hospitalcap",
#     "pollutionvol",
#     "popsize",
#     "retailturnover",
#     "livarea",

# ]
files = """parks 
schoolnum 
cliniccap 
sportsvenue 
consnewareas 
popsize 
livestock 
saldo internat 
consnewapt 
budincome 
musartschool 
foodservturnover 
goodcompanies 
invest 
factoriescap 
shoparea 
museums 
agrprod 
roadslen 
hospitals 
outflow 
foodseats 
servicesnum 
avgsalary 
harvest 
funds 
library 
cultureorg 
saldo interreg 
avgemployers 
naturesecure 
goodcompincome 
visiblecompanies 
retailturnover 
beforeschool 
theatres 
saldo reg 
docsnum 
livarea 
badcompanies """.split('\n')

files_about_mo = ["sportsvenue", "invest",

                  "popsize", "shoparea", "roadslen", "hospitals"]
# for f in files_about
#         _mo:
#     d = pd.read_csv(
#         f"/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/superdataset/features separately/{f} (allmun).csv")
#     d[["oktmo"]] = d[["oktmo"]].astype(str)
#     d["from_lo"] = [bool(re.fullmatch(r"41\d+", x)) for x in d.oktmo]
#     # d_lo = d[(d.from_lo) & (d.year == 202 2 (
#         ]
#     # print(f == f} {len(d_lo)}")
#     d = d[(d.oktmo.isin(mo_features.index)) & (
#         d.year == 2022)].drop(["from_lo"], axis=1)
#     if len(d) == 0:
#         print(f"error in {f}")
#     else:
#         mo_features = mo_features.join(d.set_index("oktmo").iloc[:, -1:], )

In [110]:
mun_districts = mo_features[pd.Series(
    mo_features.index, index=mo_features.index).apply(is_mun_district)]["name"]
mun_districts

oktmo
41603000     Бокситогорский муниципальный район
41606000        Волосовский муниципальный район
41609000         Волховский муниципальный район
41612000       Всеволожский муниципальный район
41615000         Выборгский муниципальный район
41618000         Гатчинский муниципальный район
41621000      Кингисеппский муниципальный район
41624000          Киришский муниципальный район
41625000          Кировский муниципальный район
41627000    Лодейнопольский муниципальный район
41630000      Ломоносовский муниципальный район
41633000            Лужский муниципальный район
41636000       Подпорожский муниципальный район
41639000        Приозерский муниципальный район
41642000        Сланцевский муниципальный район
41645000         Тихвинский муниципальный район
41648000         Тосненский муниципальный район
41754000                         Сосновоборский
Name: name, dtype: object

In [111]:
len(mo_features)

205

# Генерация популяции

Распределения по полу и возрасту есть только для муниципальных районов.
migration/ITMO-2/popsize/data0.xlsx

In [115]:
@dataclass
class AgeGroup:
    lo: int
    hi: int

    def __str__(self):
        return f"{self.lo}-{self.hi}"


age_groups = [
    AgeGroup(0, 0),
    AgeGroup(1, 4),
    AgeGroup(5, 9),
    AgeGroup(10, 14),
    AgeGroup(15, 19),
    AgeGroup(20, 24),
    AgeGroup(25, 29),
    AgeGroup(30, 34),
    AgeGroup(35, 39),
    AgeGroup(40, 44),
    AgeGroup(45, 49),
    AgeGroup(50, 54),
    AgeGroup(55, 59),
    AgeGroup(60, 64),
    AgeGroup(65, 69),
    AgeGroup(70, 74),
    AgeGroup(75, 79),
    AgeGroup(80, 84),
    AgeGroup(85, 89),
    AgeGroup(90, 94),
    AgeGroup(95, 99),
    AgeGroup(100, 100),
]
gender_age_dist_by_district_oktmo = {}
for mun_district_oktmo, mun_district_name_series in pd.DataFrame(mun_districts).iterrows():
    mun_district_name = mun_district_name_series.iloc[0]
    mun_district_first_name = mun_district_name.split()[0]
    try:
        df = pd.read_excel(

            "/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/popsize/data0.xlsx", sheet_name=mun_district_first_name)
    except ValueError:
        df = pd.read_excel(

            "/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/popsize/data0.xlsx", sheet_name=mun_district_first_name+" ")

    col_num = list(df.iloc[0, :]).index(YEAR)
    year_data = df.iloc[:, col_num]
    males = []
    females = []
    for i, ag in enumerate(age_groups):
        female = year_data.iloc[6+i*4]
        male = year_data.iloc[7+i*4]
        males.append(male)
        females.append(female)

    gender_age_dist = pd.DataFrame(
        {"male": males, "female": females}, index=list(map(str, age_groups)))
    gender_age_dist = gender_age_dist.fillna(gender_age_dist.mean())
    gender_age_dist_by_district_oktmo[mun_district_oktmo] = gender_age_dist
print("DONE")

DONE
